# Cat Dog Dataset com TensorFlow 2.0
# Classificação Binária

In [6]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [7]:
tf.__version__

'2.0.0-beta1'

## Dividir os dados em teste e treino

In [16]:
# transformação a ser feita nos dados  de treino e sua normalização
# ImageDataGenerator pega numa imagem e transforma aumentando a escala, esticando, virando de modo a originar mais imagens
# diferentes no conjunto de dados
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [17]:
# normalizar os dados de treino para os valores dos pixeis que variam de 0 a 255 passem a estar na escala de 0 a 1.
test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
# importar o conjunto de dados de treino(e aplicar a transformação e normalização train_datagen)
train_set = train_datagen.flow_from_directory('dataset\\training_set',
                                               target_size=(64, 64),
                                               batch_size=6,
                                               class_mode='binary')

Found 98 images belonging to 2 classes.


In [19]:
# classes existentes
label_map = (train_set.class_indices)
print(label_map)

{'cats': 0, 'dogs': 1}


In [20]:
## importar conjunto de teste (e aplicar a normalização test_datagen)
test_set = test_datagen.flow_from_directory('dataset\\test_set',
                                            target_size=(64, 64),
                                            batch_size=6,
                                            class_mode='binary')

Found 38 images belonging to 2 classes.


# Criar o Modelo Convolutional Neural Network

In [12]:
model = models.Sequential()

# 5 Convolution layers seguidas de uma pooling layer
# Imagem 64x64 a cores(3)
model.add(layers.Conv2D(filters=16, kernel_size=(3,3), padding="same", activation=tf.nn.relu, input_shape=(64,64,3)))
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", activation=tf.nn.relu))
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))         
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation=tf.nn.relu))
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation=tf.nn.relu))          
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation=tf.nn.relu))          
model.add(layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
model.add(layers.Flatten())
model.add(layers.Dense(units=512, activation="relu"))
model.add(layers.Dense(units=10, activation="relu"))     

# Não é necessário que seja a função softmax porque é um problema de classificação binário
model.add(layers.Dense(1, activation="sigmoid"))          
          


In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 16)        448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 128)        

# Compilar e Treinar o Modelo

In [27]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [28]:
model.fit_generator(
        train_set,
        steps_per_epoch=98,
        epochs=5,
        validation_data=test_set,
        validation_steps=38)

Epoch 1/5
98/98 [==============================] - 10s 102ms/step - loss: 0.6949 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/5
98/98 [==============================] - 8s 81ms/step - loss: 0.6932 - accuracy: 0.4982 - val_loss: 0.6930 - val_accuracy: 0.5096
Epoch 3/5
98/98 [==============================] - 10s 104ms/step - loss: 0.6932 - accuracy: 0.4982 - val_loss: 0.6932 - val_accuracy: 0.5048
Epoch 4/5
98/98 [==============================] - 11s 108ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.4904
Epoch 5/5
98/98 [==============================] - 9s 94ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5096


## Avaliar o Modelo

In [35]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [36]:
# fazer a previsão para um conjunto novo de dados
# Alterar o endpoint 'dataset\\single_prediction\\aaaa.jpg' para fazer as previsões para a imagem
test_image= image.load_img('dataset\\single_prediction\\cat2.jpg'
                           ,target_size =(64,64))

In [37]:
# Converter a imagem num array
test_image = image.img_to_array(test_image)

In [38]:
# For single prediction change the dimension using axis. To remove problem of batch
test_image = np.expand_dims(test_image,axis = 0)
result = model.predict(test_image)

In [39]:
## classe de cães e gatos
train_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
   prediction = 'cat'

In [40]:
# previsão
prediction

'cat'